# Using Wallaroo Shadow Deployment in Computer Vision - Part 1

In this tutorial we will explore using Wallaroo's Shadow Deployment Technology to safely improve your Object Detectors inference capabilities in a production environments.

We will configure the pipeline to use a shadow deployment where the control is the production mobilenet object detector.  We will configure a challenger resnet50 object detector.

Both object detectors will perform inference on each frame in the video and the objects they detect, their bounding boxes, and the classification confidence will be drawn on the frame.

Our goal is to study the video and its statistics to learn if the new challenger resnet50 object performs better than  the control mobilenet.


In [13]:
!pip install imutils
!pip install pytz

import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import imutils
from CVDemoUtils import CVDemo


### Initialize some Vars

Initialize the COCO Classes, meaning the classificaitons found on the images and the default width and height all images are resized to

In [14]:
wl = wallaroo.Client()

In [15]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [16]:
control =  wl.upload_model('mobilenet', 'models/mobilenet.pt.onnx')

In [17]:
challenger_list  = [ 
    wl.upload_model('resnet50', 'models/frcnn-resnet.pt.onnx')
]

In [18]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("12Gi").build()

In [19]:
pipeline = wl.build_pipeline("shadow-pp")
pipeline.add_shadow_deploy(control, challenger_list)
pipeline.deploy(deployment_config = deployment_config)

Waiting for deployment - this will take up to 45s ..... ok


name,shadow-pp
created,2022-11-08 20:04:56.582490+00:00
last_updated,2022-11-09 18:13:21.103084+00:00
deployed,True
tags,
steps,resnet50


In [20]:
time.sleep(5) # needed to allow the pipeline to settle in.
url = pipeline._deployment._url()
print(url)

http://engine-lb.shadow-pp-895:29502/pipelines/shadow-pp


In [22]:

from CVDemoUtils import CVDemo

# set the device we will be using to run the model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The set of COCO classifications
CLASSES = pickle.loads(open("models/coco_classes.pickle", "rb").read())

# Unique colors for each identified COCO class
COLORS = [CVDemo.BLUE, CVDemo.RED]

# The size the image will be resized to
width = 640
height = 480

# Only objects that have a confidence > confidence_target will be displayed on the image
confidence_target = 0.75

cvDemo = CVDemo(CLASSES,COLORS, DEVICE)

### Detect and Classify Objects in Video using Wallaroo Shadow Deployment

TBD

In [23]:
cvDemo = CVDemo(CLASSES,COLORS, DEVICE)

# Only objects that have a confidence > confidence_target will be displayed on the image
input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-shadow-part-1-inferenced.mp4"
cur_ws = wl.get_current_workspace()
config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 25, # Frames per second
    'pipeline' : pipeline,
    'control-model' : control,
    'challenger-model-list' : challenger_list,
    'endpoint-url' : url, # the pipelines rest api endpoint url
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'max-frames' : 300, # the # of frames to capture in the output video
    'skip-frames' : 100, # the # of frames to capture in the output video
    'confidence-target' : 0.90, # only display bounding boxes with confidence > provided #
    'inference': 'WALLAROO_SDK',
}
cvDemo.DEBUG = False
cvDemo.detectAndClassifyObjectsInVideoUsingShadowDeployment(config)
print("We are done.")



11-09-2022 18:13:33.977844 Start Time:01:13:33 PM
11-09-2022 18:13:33.977934 Skipping [100] frames
11-09-2022 18:13:33.977946 Captureing up to [300] frames
11-09-2022 18:13:33.983652 Video Properties
11-09-2022 18:13:33.983725    format:0.0
11-09-2022 18:13:33.983736    fourcc:828601953.0
11-09-2022 18:13:33.983744    mode:0.0
11-09-2022 18:13:33.983752    buffer:0.0
11-09-2022 18:13:33.983760    width:1280.0
11-09-2022 18:13:33.983767    height:720.0
11-09-2022 18:13:33.983774    fps:25.0
11-09-2022 18:13:33.983781    frame count:16054.0
11-09-2022 18:13:39.802007 Frame:1 Read: 0.0011 Inf: 5.4701 Onnx: 0.0000 Draw: 0.0057 Total: 5.5512
11-09-2022 18:13:44.869342 Frame:2 Read: 0.0012 Inf: 5.0188 Onnx: 0.0000 Draw: 0.0063 Total: 5.0672
11-09-2022 18:13:50.218690 Frame:3 Read: 0.0011 Inf: 5.2998 Onnx: 0.0000 Draw: 0.0057 Total: 5.3492
11-09-2022 18:13:55.516640 Frame:4 Read: 0.0011 Inf: 5.2498 Onnx: 0.0000 Draw: 0.0056 Total: 5.2978
11-09-2022 18:14:00.807651 Frame:5 Read: 0.0011 Inf: 5.

In [25]:
#pipeline.undeploy()
for d in wl.list_deployments():
    d.undeploy()
    

 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok
 ok


### Conclusion

Notice the difference in the control confidence and the challenger confidence.  Clearly we can see in this example the Control resnet50 model is performing better than the mobilenet.  This is likely due to the fact that frcnn resnet50 model is a 2 stage object detector vs the mobilenet is a single stage detector.

In the next example we will take this to another level and see what happens when we use shadow deployment in a video in order to achieve the best of both worlds.  Highest accuracy with the best performance.